# 16. Classes and Functions
> Object-Oriented Programming 2

- toc: true
- badges: true
- comments: true
- categories: [python]
- author: Zhi Li
- sticky_rank: 16




Now that we know how to create new types, the next step is to write functions that take programmer-defined objects as parameters and return them as results.

## Time

In [ ]:
class Time:
    """Represents the time of day.
       
    attributes: hour, minute, second
    """

time = Time()
time.hour = 1
time.minute = 31
time.second = 30

### ***Exercise 01***

1. write a function called <code>print_time</code> that takes a Time object and prints it in the form <code>hour:minute:second</code>. Hint: <code>'f{foo:02}'</code> prints an integer variable, `foo`, using at least two digits, including a leading zero if necessary.
2. Write a boolean function called <code>is_after</code> that takes two Time objects, <code>t1</code> and <code>t2</code>, and returns <code>True</code> if <code>t1</code> follows <code>t2</code> chronologically and <code>False</code> otherwise. 
    **Challenge**: don’t use an <code>if</code> statement.

## Prototyping and patching

In the next few sections, we’ll write two functions that add time values. They demonstrate two kinds of functions: pure functions and modifiers. They also demonstrate a development plan I’ll call **prototype and patch**, which is a way of tackling a complex problem by starting with a simple prototype and incrementally dealing with the complications.

In [ ]:
# Here is a simple prototype of add_time:
def add_time(t1, t2):
    sum = Time()
    sum.hour = t1.hour + t2.hour
    sum.minute = t1.minute + t2.minute
    sum.second = t1.second + t2.second
    return sum

There might be some problems:

In [ ]:
start = Time()
start.hour = 9
start.minute = 45
start.second =  0

duration = Time()
duration.hour = 1
duration.minute = 35
duration.second = 0

done = add_time(start, duration)
print_time(done)

### ***Exercise 02***
Let's improve <code>add_time</code>.

### Modifier

The above function is called a **pure function** because it does not modify any of the objects passed to it as arguments and it has no effect, like displaying a value or getting user input, other than returning a value. 

Sometimes it is useful for a function to modify the objects it gets as parameters. In that case, the changes are visible to the caller. Functions that work this way are called **modifiers**. For example:


In [ ]:
def increment(time, seconds):
    time.second += seconds

    if time.second >= 60:
        time.second -= 60
        time.minute += 1

    if time.minute >= 60:
        time.minute -= 60
        time.hour += 1

*Question:* Is this function correct? What happens if seconds is much greater than sixty? 

Anything that can be done with **modifiers** can also be done with **pure functions**. In fact, some programming languages only allow pure functions. There is some evidence that programs that use pure functions are faster to develop and less error-prone than programs that use modifiers. But modifiers are convenient at times, and functional programs tend to be less efficient.

In general, I recommend that you write pure functions whenever it is reasonable and resort to modifiers only if there is a compelling advantage. This approach might be called a **functional programming style**. 

### ***Exercise 03***
Write a "**pure**" version of <code>increment</code> that creates and returns a new Time object rather than modifying the parameter.

## Prototyping versus planning


The development plan above is called “**prototype and patch**”. For each function, we wrote a prototype that performed the basic calculation and then tested it, patching errors along the way.

This approach can be effective, especially if you don’t yet have a deep understanding of the problem. But incremental corrections can generate code that is unnecessarily complicated—since it deals with many special cases—and unreliable—since it is hard to know if you have found all the errors.

An alternative is **designed development**, in which high-level insight into the problem can make the programming much easier. In this case, the insight is that a Time object is really a three-digit number in base 60 (see http://en.wikipedia.org/wiki/Sexagesimal.) The second attribute is the “ones column”, the minute attribute is the “sixties column”, and the hour attribute is the “thirty-six hundreds column”.

When we wrote `add_time`, we were effectively doing addition in base 60, which is why we had to carry from one column to the next. 

This observation suggests another approach to the whole problem — we can convert Time objects to integers and take advantage of the fact that the computer knows how to do integer arithmetic.

Here is a function that converts Times to integers:

In [ ]:
def time_to_int(time):
    minutes = time.hour * 60 + time.minute
    seconds = minutes * 60 + time.second
    return seconds

And here is a function that converts an integer to a Time (recall that <code>divmod</code> divides the first argument by the second and returns the quotient and remainder as a tuple). 

In [ ]:
def int_to_time(seconds):
    time = Time()
    minutes, time.second = divmod(seconds, 60)
    time.hour, time.minute = divmod(minutes, 60)
    return time

Once you are convinced they are correct, you can use them to rewrite add_time:

In [ ]:
def add_time_2(t1, t2):
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

This version is shorter than the original, and easier to verify.

It is also easier to add features later. For example, imagine subtracting two Times to find the duration between them. The naive approach would be to implement subtraction with borrowing. Using the conversion functions would be easier and more likely to be correct. 

### ***Exercise 4***

Write a function called <code>substract_time</code> that takes two Time objects returns find the duration between them.



## Error handling



In [ ]:
def valid_time(time):
    if time.hour < 0 or time.minute < 0 or time.second < 0:
        return False
    if time.minute >= 60 or time.second >= 60:
        return False
    return True


def add_time_2(t1, t2):
    if not valid_time(t1) or not valid_time(t2):
        raise ValueError('invalid Time object in add_time')
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

Or you could use an **assert statement**, which checks a given invariant and raises an exception if it fails: 

In [ ]:
def add_time(t1, t2):
    assert valid_time(t1) and valid_time(t2)
    seconds = time_to_int(t1) + time_to_int(t2)
    return int_to_time(seconds)

<code>assert</code> statements are useful because they distinguish code that deals with normal conditions from code that checks for errors.

### ***Exercise 05***

Write a function called <code>mul_time</code> that takes a Time object and a number and returns a new Time object that contains the product of the original Time and the number.

Then use <code>mul_time</code> to write a function that takes a Time object that represents the finishing time in a race, and a number that represents the distance, and returns a Time object that represents the average pace (time per mile). 

### ***Exercise 06***


The <code>datetime</code> module provides <code>time</code> objects that are similar to the Time objects in this chapter, but they provide a rich set of methods and operators. Read the documentation at http://docs.python.org/3/library/datetime.html.

1. Use the <code>datetime</code> module to write a program that gets the current date and prints the day of the week.
2. Write a program that takes a birthday as input and prints the user’s age and the number of days, hours, minutes and seconds until their next birthday.
3. For two people born on different days, there is a day when one is twice as old as the other. That’s their Double Day. Write a program that takes two birthdays and computes their Double Day.
4. For a little more challenge, write the more general version that computes the day when one person is n times older than the other. 